In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from skimage.segmentation import morphological_chan_vese
from skimage.filters import gaussian
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt

IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
MASK_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Masks"
df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")

def segment_optic_disc(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    smoothed = gaussian(gray, sigma=1)
    segmented_mask = morphological_chan_vese(smoothed, iterations=100, init_level_set="checkerboard")
    return segmented_mask

def load_data(df, image_size=(512, 512)):
    images, labels = [], []
    
    for _, row in df.iterrows():
        img_name, label = row['imageID'], row['binaryLabels']
        img_path = os.path.join(IMAGES_FOLDER, img_name)
        
        if not os.path.exists(img_path):
            continue

        # Load and preprocess image
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, image_size)
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Image processing
        edges = cv2.Laplacian(gray, cv2.CV_64F)
        edges = cv2.convertScaleAbs(edges) 
        sharpened = cv2.addWeighted(gray, 1.5, edges, -0.5, 0)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(sharpened)
        normalized = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
        
        images.append(normalized)
        labels.append(label)
    
    return np.array(images), np.array(labels)

def glcm_features(image):
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    return contrast, correlation

def compute_cslbp(image):
    radius, n_points = 1, 8
    return local_binary_pattern(image, n_points, radius, method='uniform')

def extract_features(images):
    feature_vectors = []
    for image in images:
        contrast, correlation = glcm_features(image)
        lbp_hist = np.histogram(compute_cslbp(image).ravel(), bins=256, range=(0, 256))[0]
        lbp_hist = lbp_hist / np.sum(lbp_hist)
        feature_vectors.append(np.hstack((contrast, correlation, lbp_hist)))
    return np.array(feature_vectors)

def train_svm(features, labels):
    svm = SVC(kernel='linear', probability=True)
    svm.fit(features, labels)
    return svm

images, labels = load_data(df)
if images.size == 0:
    raise Exception("No images loaded. Check dataset paths.")

features = extract_features(images)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

svm_model = train_svm(X_train, y_train)
print("SVM Model Trained Successfully")

accuracy = svm_model.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


SVM Model Trained Successfully
Test Accuracy: 0.7108
